In [1]:
import bioformats
import deepdish as dd
import h5py
import javabridge
import numpy as np
import os.path
from pyprind import prog_percent
import SimpleITK as sitk
import tables
import time
from xml.etree import ElementTree as ETree

# https://stackoverflow.com/questions/40845304/runtimewarning-numpy-dtype-size-changed-may-indicate-binary-incompatibilityV
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import deepdish as dd
import argparse
import numpy as np
import javabridge as jv
import bioformats as bf
from pyprind import prog_percent
from xml import etree as et
from queue import Queue
from threading import Thread, Lock
from enum import Enum
import torch as th
from torch.autograd import Variable
from torch.nn.functional import grid_sample
from tqdm import tqdm_notebook
import h5py
import os
from skimage.feature import blob_log
from skimage.draw import circle
from skimage.transform import AffineTransform, warp
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import matplotlib.cm as mplColorMap
import ipywidgets as widgets
#import ipyvolume as ipv
from scipy.interpolate import RegularGridInterpolator

SPACING_ZBB = (0.798, 0.798, 2)
SPACING_JAKOB = (0.7188675, 0.7188675, 10)
SPACING_JAKOB_HQ = (0.7188675, 0.7188675, 1)


def lif_get_metas(fn):
    md = bioformats.get_omexml_metadata(fn)  # Load meta data
    mdroot = ETree.fromstring(md)  # Parse XML
    #    meta = mdroot[1][3].attrib # Get relevant meta data
    metas = list(map(lambda e: e.attrib, mdroot.iter('{http://www.openmicroscopy.org/Schemas/OME/2016-06}Pixels')))

    return metas

def lif_find_timeseries(fn):
    metas = lif_get_metas(fn)

    meta = None
    img_i = 0
    for i, m in enumerate(metas):
        if int(m['SizeT']) > 1:
            meta = m
            img_i = i

    if not meta:
        raise ValueError('lif does not contain an image with sizeT > 1')

    return img_i

def start_jvm():
    javabridge.start_vm(class_path=bioformats.JARS)

    log_level = 'ERROR'
    # reduce log level
    """
    rootLoggerName = javabridge.get_static_field("org/slf4j/Logger", "ROOT_LOGGER_NAME", "Ljava/lang/String;")
    rootLogger = javabridge.static_call("org/slf4j/LoggerFactory", "getLogger",
                                        "(Ljava/lang/String;)Lorg/slf4j/Logger;", rootLoggerName)
    logLevel = javabridge.get_static_field("ch/qos/logback/classic/Level", log_level, "Lch/qos/logback/classic/Level;")
    javabridge.call(rootLogger, "setLevel", "(Lch/qos/logback/classic/Level;)V", logLevel)
    """


def lif_open(fn):
    start_jvm()
    ir = bioformats.ImageReader(fn)

    return ir

def lif_read_stack(fn):
    ir = lif_open(fn)
    img_i = lif_find_timeseries(fn)
    shape, spacing = get_shape(fn, img_i)

    stack = np.empty(shape, dtype=np.uint16)

    # Load the whole stack...
    for t in prog_percent(range(stack.shape[0])):
        for z in range(stack.shape[1]):
            stack[t, z] = ir.read(t=t, z=z, c=0, series=img_i, rescale=False)

    return stack, spacing

def get_shape(fn, index=0):
    """

    :param fn: image file
    :return: shape of that file
    """
    in_ext = os.path.splitext(fn)[1]

    if in_ext == '.h5':
        """
        f = tables.open_file(fn)
        return f.get_node('/stack').shape
        """
        img = load(fn)
        return img.shape
    elif in_ext == '.nrrd':
        img = load(fn)
        return img.shape
    elif in_ext == '.lif':
        metas = lif_get_metas(fn)
        meta = metas[index]

        shape = (
            int(meta['SizeT']),
            int(meta['SizeZ']),
            int(meta['SizeY']),
            int(meta['SizeX']),
        )
        order = meta['DimensionOrder']
        spacing = tuple([float(meta['PhysicalSize%s' % c]) for c in 'XYZ'])

        return shape, spacing

    else:
        raise UnsupportedFormatException('Input format "' + in_ext + '" is not supported.')


def __sitkread(filename):
    img = sitk.ReadImage(filename)
    spacing = img.GetSpacing()
    return sitk.GetArrayFromImage(img), spacing


def __sitkwrite(filename, data, spacing):
    img = sitk.GetImageFromArray(data)
    img.SetSpacing(spacing)
    sitk.WriteImage(img, filename)

def save(fn, data, spacing):
    out_ext = os.path.splitext(fn)[1]

    if out_ext == '.nrrd':
         __sitkwrite(fn, data, spacing)
    elif out_ext == '.h5':
        """
        with tables.open_file(fn, mode='w') as f:
            f.create_array('/', 'stack', data.astype(np.float32))
            f.close()
        """
        __sitkwrite(fn, data, spacing)
    else:
         raise UnsupportedFormatException('Output format "' + out_ext + '" is not supported.')

            
class Pyfish:

    def __init__(self, file_id, save_path="", align_to_frame=0, use_gpu=True, max_displacement=300, thread_count=4):
        #self.lif_file_path = lif_file_path
        self.file_id=file_id
        self.align_to_frame = align_to_frame #un frami ke nesbat be un baghie axaro align mikonim
        self.use_gpu = use_gpu
        self.max_displacement = max_displacement
        self.thread_count = thread_count

        self._start_lif_reader()
        print('Number of timepoints: ',self.n_timepoints)
        self.frame_shape=(self.n_timepoints, 1024, 1024)
        #self.frame_shape=


    # start lif reader
    # set image stack shape
    # lif files can contain multiple stacks so we pick the index of one with more than one frames in lif_stack_idx
    def _start_lif_reader(self):
        jv.start_vm(class_path=bf.JARS)

        log_level = 'ERROR'
        
        if self.file_id[1]=='control':
        
            file_path='//ZMN-HIVE/User-Data/Maria/control/fish'+self.file_id[0]+'_6dpf_medium.lif'
        elif self.file_id[1]=='amph':
            file_path='//ZMN-HIVE/User-Data/Maria/stimulus/fish'+self.file_id[0]+'_6dpf_amph.lif'
        

        self.ir = bf.ImageReader(file_path, perform_init=True)

        mdroot = et.ElementTree.fromstring(bf.get_omexml_metadata(file_path))
        mds = list(map(lambda e: e.attrib, mdroot.iter('{http://www.openmicroscopy.org/Schemas/OME/2016-06}Pixels')))

        # lif can contain multiple images, select one that is likely to be the timeseries
        self.metadata = None
        self.lif_stack_idx = 0
        for idx, md in enumerate(mds):
            if int(md['SizeT']) > 1:
                self.lif_stack_idx = idx
                self.metadata      = md
        if not self.metadata: raise ValueError('lif does not contain an image with sizeT > 1')
        self.n_timepoints=int(md['SizeT'])        

    def read_frame(self, z):
        frame = np.empty(self.frame_shape, dtype=np.uint16)
        for t in range(self.n_timepoints):
            frame[t] = self._read_plane(t, z)
        return frame

    def _read_plane(self, t, z):
        return self.ir.read(t=t, z=z, c=0, series=self.lif_stack_idx, rescale=False)
    
    def save_hdf5(self):
        start=time.time()
        hdf5_file = np.zeros((self.n_timepoints,21,1024,1024),dtype='float32')
        for z in range(21):
            print('working on plane: ', z)
            dat=self.read_frame(z)
            hdf5_file[:,z,:,:]=dat
        end=time.time()
        SPACING_ZBB = (0.798, 0.798, 2)
        SPACING_JAKOB = (0.7188675, 0.7188675, 10)
        SPACING_JAKOB_HQ = (0.7188675, 0.7188675, 1)
        fn='//ZMN-HIVE/User-Data/Maria/hdf5_conversion/fish'+self.file_id[0]+'_6dpf_medium.h5'
        #save(fn, hdf5_file, SPACING_JAKOB)
        with h5py.File(fn, 'w') as f:
            dset = f.create_dataset("data", data=hdf5_file)
        print('Time taken to save: ', end-start)


In [2]:
file_ids=[['11','control']]  
for j in file_ids:
    data_processor=Pyfish(j)
    data_processor.save_hdf5()

Number of timepoints:  1800
working on plane:  0
working on plane:  1
working on plane:  2
working on plane:  3
working on plane:  4
working on plane:  5
working on plane:  6
working on plane:  7
working on plane:  8
working on plane:  9
working on plane:  10
working on plane:  11
working on plane:  12
working on plane:  13
working on plane:  14
working on plane:  15
working on plane:  16
working on plane:  17
working on plane:  18
working on plane:  19
working on plane:  20
Time taken to save:  1224.7974812984467
